In [1]:
import pandas as pd
import glob

import colorlover as cl

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import plotly
plotly.tools.set_credentials_file(username='iverson3', api_key='gE3StFoPy53qjW3ReF68')

## Data processing

In [7]:
# read metadata
trbv = pd.read_csv("./segments/segments.wt.V.txt", sep="\t")
trbv.head()

,sample_id,time_point,population,..filter..,TRBV1-1,TRBV10-1,TRBV10-2,TRBV10-3,TRBV11,TRBV11-1,...,TRBV7-4,TRBV7-5,TRBV7-6,TRBV7-7,TRBV7-8,TRBV7-9,TRBV8,TRBV8-2,TRBV9-1,TRBVA-or9_2
0,3_IL5,3,IL5,conv:ImmunoSeq,0.000146,0.001268,0.012049,0.056341,0.000634,0.000927,...,0.000585,0.000537,0.001707,0.000390,0.004683,0.009610,0.000000,0.000049,0.041220,0.000098
1,1_IL10,1,IL10,conv:ImmunoSeq,0.006098,0.003049,0.006098,0.022866,0.000000,0.000000,...,0.003049,0.000000,0.004573,0.000000,0.022866,0.018293,0.000000,0.000000,0.022866,0.000000
2,2_IL5,2,IL5,conv:ImmunoSeq,0.000159,0.001061,0.005463,0.036385,0.000318,0.000265,...,0.001114,0.000212,0.002228,0.000159,0.004349,0.009812,0.000000,0.000000,0.042113,0.000000
3,1_IL5,1,IL5,conv:ImmunoSeq,0.000506,0.001348,0.009058,0.037961,0.000211,0.000674,...,0.000590,0.000211,0.001180,0.000463,0.005814,0.008806,0.000042,0.000000,0.047735,0.000042
4,6_IL10,6,IL10,conv:ImmunoSeq,0.002423,0.005654,0.007270,0.017771,0.000000,0.000000,...,0.005654,0.000000,0.004847,0.001616,0.017771,0.019386,0.000000,0.001616,0.032310,0.000000


In [8]:
# remove unnecessary columns
trbv.drop(["..filter..", "TRBVA-or9_2", "sample_id"], axis=1, inplace=True)
trbv.head()

,time_point,population,TRBV1-1,TRBV10-1,TRBV10-2,TRBV10-3,TRBV11,TRBV11-1,TRBV11-2,TRBV11-3,...,TRBV7-3,TRBV7-4,TRBV7-5,TRBV7-6,TRBV7-7,TRBV7-8,TRBV7-9,TRBV8,TRBV8-2,TRBV9-1
0,3,IL5,0.000146,0.001268,0.012049,0.056341,0.000634,0.000927,0.006488,0.049610,...,0.002829,0.000585,0.000537,0.001707,0.000390,0.004683,0.009610,0.000000,0.000049,0.041220
1,1,IL10,0.006098,0.003049,0.006098,0.022866,0.000000,0.000000,0.018293,0.042683,...,0.000000,0.003049,0.000000,0.004573,0.000000,0.022866,0.018293,0.000000,0.000000,0.022866
2,2,IL5,0.000159,0.001061,0.005463,0.036385,0.000318,0.000265,0.006312,0.028694,...,0.001803,0.001114,0.000212,0.002228,0.000159,0.004349,0.009812,0.000000,0.000000,0.042113
3,1,IL5,0.000506,0.001348,0.009058,0.037961,0.000211,0.000674,0.006151,0.039141,...,0.002696,0.000590,0.000211,0.001180,0.000463,0.005814,0.008806,0.000042,0.000000,0.047735
4,6,IL10,0.002423,0.005654,0.007270,0.017771,0.000000,0.000000,0.027464,0.016963,...,0.006462,0.005654,0.000000,0.004847,0.001616,0.017771,0.019386,0.000000,0.001616,0.032310


In [9]:
# total v segments
print(len(trbv.columns)-2)

72


In [16]:
# plot IL5 and IL10 separately
il_5 = trbv.loc[trbv["population"]=="IL5"].drop("population", axis=1)
il_10 = trbv.loc[trbv["population"]=="IL10"].drop("population", axis=1)

In [21]:
# set time point as index
il_5 = il_5.sort_values("time_point").set_index("time_point")
il_10 = il_10.sort_values("time_point").set_index("time_point")

In [25]:
# rearrange columns by v names
def sort_v_name(v):
    v = v[4:].split("-")
    if len(v) == 1:
        return (int(v[0]), 0)
    else:
        return (int(v[0]), int(v[1]))

In [28]:
cols = trbv.columns.tolist()[2:]
cols = sorted(cols, key=sort_v_name)
il_5 = il_5[cols]
il_10 = il_10[cols]

In [29]:
# 80 colors
paired = cl.scales['11']['qual']['Set3']
paired80 = cl.interp(paired, 80)

# each v segment gets a unique color
v_colors = {cols[i]:paired80[i] for i in range(len(cols))}

In [30]:
import plotly.plotly as py
import plotly.graph_objs as go

## Plot the usage of V segments for each population in each individual donor. Please note that each V segment is represented by the same color in all the plots.

In [33]:
data = []

for col in cols:
    trace = go.Bar(
        x=il_5.index.values,
        y=il_5[col],
        name=col,
        marker=dict(color=v_colors[col]),
    )
    data.append(trace)

layout = go.Layout(
    title="IL-5",
    barmode='stack',
    legend=dict(traceorder='normal'),
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="IL-5")

In [34]:
data = []

for col in cols:
    trace = go.Bar(
        x=il_10.index.values,
        y=il_10[col],
        name=col,
        marker=dict(color=v_colors[col]),
    )
    data.append(trace)

layout = go.Layout(
    title="IL-10",
    barmode='stack',
    legend=dict(traceorder='normal'),
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="IL-10")